# Visualizatie van de toename/afname van de richness per meet object

## Imports

In [ ]:
#Auteur: Marjolein Spijkerman

import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import Point
from scipy.stats import linregress

## Code inlezen en klaarzetten

In [ ]:
#op basis van het Excel bestand: 
# https://github.com/CopEDA/hackathon_juni_2025/blob/main/hackathon_Jeanine/Macrofyten_Hackathon2.csv
df_rich = gpd.read_file("Macrofyten_Hackathon2.csv")

#De coördinaten moeten in graden worden gezet in de geometry kolom
geometry = [Point(xy) for xy in zip(df_rich['X'], df_rich['Y'])]
gdf_rich = gpd.GeoDataFrame(df_rich, geometry=geometry, crs="EPSG:28992")
gdf_rich = gdf_rich.to_crs("EPSG:4326")

#Een aantal kolommen waren opgeslagen als string ipv getal
gdf_rich['Richness'] = pd.to_numeric(gdf_rich['Richness'], errors='coerce')
gdf_rich['Jaar'] = pd.to_numeric(gdf_rich['Jaar'], errors='coerce')

In [ ]:
#Korte visualizatie van de dataframe
gdf_rich.head()

,MeetObject,Naam,GEMEENTE,X,Y,Jaar,Maand,Instanties,Netwerken,Landgebruik,Vaarwater,Gebied_Type,Richness,geometry
0,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,2013,7,NA,EBEO_H&W,Stad,Ja,Boezem,19,POINT (4.34334 52.0205)
1,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,2019,8,RTR,EBEO_H&W,Stad,Ja,Boezem,5,POINT (4.34334 52.0205)
2,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,2022,7,HHE,EBEO_H&W,Stad,Ja,Boezem,1,POINT (4.34334 52.0205)
3,OW004-000,Kromme Zweth_Noordlierweg afbuiging,Westland,77028,445194,2014,7,NA,EBEO_MD,Glas,Ja,Boezem,34,POINT (4.25201 51.98966)
4,OW004-000,Kromme Zweth_Noordlierweg afbuiging,Westland,77028,445194,2023,8,FBT,EBEO_MD,Glas,Ja,Boezem,17,POINT (4.25201 51.98966)


## Het berekenen van de slopes

In [29]:
#We gaan door alle meetobjecten heen
objects = gdf_rich['MeetObject'].unique().tolist()

#We maken een lijst met slopes die even lang is als het originele dataframe. 
slopes = []

#voor elk object berekenen we de slope
for obj in objects:
    filter_df = gdf_rich[gdf_rich['MeetObject'] == obj] #filter op object
    filter_df = filter_df.sort_values('Jaar') #jaren op chronologische volgorde filteren
    vals = filter_df['Richness'].tolist() #lijst met richness waardes die bij dit object horen
    years = filter_df['Jaar'].tolist() #lijst met jaren (chronologisch)
    x = np.array(years) #we moeten de jaren opslaan as np array
    slope, intercept, r_value, p_value, std_err = linregress(x, vals) #linregress returnt de slope en nog wat andere waardes
    
    #Om de lengte van de lijst gelijk te houden aan de dataframe, slaan we de slope net zo vaak op als dat het 
    #meetobject voorkomt in de originele dataframe. We slaan later deze lijst op als een nieuwe kolom in de dataframe!
    slopes.extend([slope] * len(filter_df)) 

    #optioneel, print de waardes om te zien wat er gebeurt :)
    '''
    if slope > 0:
        print(x, vals)
        print(slope, " Stijgend")
    elif slope < 0:
        print(x, vals)
        print(slope, " Dalend")
    else:
        print("blijft exact hetzelfde")
    '''

#voeg de slopes lijst toe aan de dataframe
gdf_rich['slope'] = slopes

## De visualizatie

In [30]:
gdf_rich.explore(
    column='slope',         # Kolom waarop we filteren
    cmap='viridis',         # Kleurschema van de stippen
    legend=True,            # Legenda
    tiles='CartoDB positron',  # De stijl van de kaart. Als je deze weghaald is de kaart volledig in kleur
    marker_kwds={'radius': 5}, # Grootte van de punten op de kaart
)